# Demystifying the `IO ()` action.


What is the `IO ()` action and how does it play with a pure functional system?

`()` is known as the unit type, or the empty tuple. Because it's empty, it contains nothing useful! 
A function that returns `()` is a function that returns no useful information.

Now, imagine `IO a` is a container that contains hidden data and public data. The `a` is the public data. We don't know what the hidden data is.
A function that returns `IO ()` is a function that returns no useful public data, but might contain important hidden data, such as what to read/write to the console!

## `MyIO`

Here's an example `MyIO`, which is similar to `IO`. It has public data like `IO`, and hidden data. But our hidden data is simply a `String`.

Understanding the below instance definitions isn't required, these are provided so we can use `MyIO` like we use `IO` later in this notebook.

In [42]:
import Control.Applicative 

newtype MyIO a = MyIO { runIO :: (a, String) }

instance Functor MyIO where
    fmap f a = let (a', s) = runIO a in MyIO (f a', s) 

instance Applicative MyIO where
  pure a = MyIO (a, "")
  (<*>) fa b = let (fa', s) = runIO fa 
                   (b', s') = runIO b
                in MyIO (fa' b', s ++ s')

instance Monad MyIO where
     (>>=) a f =  let (a', s)  = runIO a
                      (b', s') =  runIO $ f a'
                   in MyIO (b', s ++ s')

In `IO` we have `getLine` and `writeStrLine` for interacting with the user.

In `MyIO` we have `myGetLine` and `myWriteLine`. We don't interact with the user, but we can pretend!

- `myGetLine` always returns the string `World` wrapped in a `MyIO`.
- `myWriteLine` sets the public data to `()` but modifies the hidden data underneath which we normally can't access.

We can call `runIO` to inspect the contents of `MyIO`. This effectively "executes" the `MyIO` and is similar to how haskell calls `main` under the hood!

In [44]:

myGetLine :: MyIO String
myGetLine = pure "World"

myWriteLine :: String -> MyIO ()
myWriteLine s = MyIO ((), s)

myMain :: MyIO ()
myMain = do 
    a <- myGetLine 
    myWriteLine $ "Hello " ++ a

runIO myMain


((),"Hello World")

Notice that we get a tuple containing the public data `()`, and the hidden data "Hello World".

Next, multiple invocations of `myWriteLine` will join together all the contents in the hidden data.

This is because `do` blocks in the Haskell language join together by calling `(>>=)` (defined above).

So each call will update the hidden data by joining together the old hidden data and new hidden data.

In [39]:
multipleWrites :: MyIO ()
multipleWrites = do
    myWriteLine "Hello World"
    myWriteLine "How are you?"

runIO multipleWrites


((),"Hello WorldHow are you?")

Some more advanced examples of reading integers and floats from input, and returning values other than `()`:

In [40]:
myReadLn :: Read a => MyIO a 
myReadLn = pure (read "3")

getInt :: MyIO Int
getInt = do
    myWriteLine "Input an integer: "
    myReadLn

runIO getInt

getFloat :: MyIO Float
getFloat = do
    myWriteLine "Input a float: "
    myReadLn

runIO getFloat

(3,"Input an integer: ")

(3.0,"Input a float: ")